# SpotRM API (Python)

# API Manual, including examples 
 
BSD 3-Clause License<br>
Copyright (c) 2020/2021 Awametox AB<br>
You should have received a copy of the BSD 3-Clause License
along with this script; <br>
if not, see <https://github.com/Awamegit/SpotRM_API_examples/blob/master/LICENSE>

### 1. Introduction & Essential set-up
SpotRM on the Web comes with an API that is included in the subsription. It should allow for individual mining of your data vs the SpotRM database. E.g. different views, integration into your cheminformatics system, high-throuput analysis, etc, etc. <br>
Here we show the principle use of the API with some basic and some more advanced examples, based on Python in this Jupyter Notebook.

*First of, some minimum, essential library imports:*

In [1]:
import requests
import json
from requests.auth import HTTPBasicAuth

*Additional (optional)libraries:*

In [2]:
import os
import rdkit

*repeating variables for get and put* <br>
Furthermore authentication access. There are two options here: either clear text (not recommendable). Use:<br>
`authData = HTTPBasicAuth('my_email@somewhere.com', 'my_super_secure_PassW0rd!` <br>
Here we make use of the username and password stored in our OS environment. In this case called *STDEMAIL* and *SPOTRM_PWD*.<br>
Every API call later made will contain authentication info.

In [9]:
BASEURL = "https://www.spotrm.com/api/v1"
headers = {"content-type": "application/json"}
authData = HTTPBasicAuth(os.environ["STDEMAIL"], os.environ["SPOTRM_PWD"])

You can also create a token for authenticating yourself:

In [7]:
url = BASEURL + "/tokens/"
response = requests.post(url, auth=authData)
token = json.loads(response.text)["token"]

Instead of using *authData* as `auth`, you can now use the token instead as an additional layer of security.

### 2. Get help on API via API
This section describes a simple and minimalistic API call requesting */help*. This will list all possible API calls. Username and Password are used for the authentication. <br>
The returned data is always formatted as JSON. In Python this can be dealt with as a dictionary. Note that parts of the entrie in such a dictionary may contain a list, particularly *References* which in turn may contain a dictionary. <br>
You can use the standard http(s) error code list to check for status errors. Code 200 stands for no error (i.e. ok).

In [5]:
response = requests.get(BASEURL + "/help", auth=authData)
respDict = json.loads(response.text)
if response.status_code == 200:
    for key in respDict:
        print(key + ": " + respDict[key])    
else:
    print("There was an error: " + json.loads(response.text)["message"])

/api/v1/get/drug/id/<id>: Find all data on a drug with the supplied ID
/api/v1/get/drug/smarts_id/<smarts_id>: Get drug records linked to given smarts_id
/api/v1/get/image/inchi: Generate highlighted image of InChI where it matches <smarts_id>
/api/v1/get/image/molfile: Generate highlighted image of molfile where it matches <smarts_id>
/api/v1/get/image/smiles: Generate highlighted image of <smiles> where it matches <smarts_id>
/api/v1/help: Print available functions and their documentation.
/api/v1/search/drug/substructure/inchi: Find all drugs containing your query structure posted as InChI as a substructure
/api/v1/search/drug/substructure/molfile: Find all drugs containing your query structure posted as Molfile as a substructure
/api/v1/search/drug/substructure/smiles: Find all drugs containing your query posted as smiles as a substructure
/api/v1/search/drug/text/<text>: Find all drugs with <text> in their records
/api/v1/search/smarts/inchi: Find all SMARTS which match the posted

### 3. Examples of API calls
Following here are some examples of API calls. Very simplistic, including means of output/working with the JSON format. The latter really depends on your use case.
#### 3.1  Here an example of checking for a drug ID (ID = SpotRM database specific):
This is a simple example where you would use the SPOT RM database ID of a drug (known/retrieved by other means, see further below). <br>Note: no error checking is done in this example when looking at ID #3.<br>
Also note how *References* is a list of dictionaries within the dictionary output.

In [16]:
lookup_an_ID = requests.get(BASEURL + "/get/drug/id/3", auth=authData)
respDict = json.loads(lookup_an_ID.text)

print("\n---- JSON output raw / unformatted -----")
print(respDict)
    
print("\n---- JSON output formatted 1 -----")
print(json.dumps(respDict, indent = 2))
    
print("\n---- JSON output formatted 2 -----")
for key, value in respDict.items():
    print(key, " = ", value)  


---- JSON output raw / unformatted -----
{'ChemName': '1,5-dimethyl-2-phenyl-pyrazol-3-one', 'ColourCode': 'Orange', 'DrugName': 'phenazone', 'MarketStatus': 'On the market (2020)', 'References': [{'Ref': 'aminophenazone.pdf', 'RefType': 'Mgph'}], 'Synonym': 'antipyrine'}

---- JSON output formatted 1 -----
{
  "ChemName": "1,5-dimethyl-2-phenyl-pyrazol-3-one",
  "ColourCode": "Orange",
  "DrugName": "phenazone",
  "MarketStatus": "On the market (2020)",
  "References": [
    {
      "Ref": "aminophenazone.pdf",
      "RefType": "Mgph"
    }
  ],
  "Synonym": "antipyrine"
}

---- JSON output formatted 2 -----
ChemName  =  1,5-dimethyl-2-phenyl-pyrazol-3-one
ColourCode  =  Orange
DrugName  =  phenazone
MarketStatus  =  On the market (2020)
References  =  [{'Ref': 'aminophenazone.pdf', 'RefType': 'Mgph'}]
Synonym  =  antipyrine


In [25]:
# a single value only (of above example)
from termcolor import colored
outCol = respDict['ColourCode']
if outCol == 'Orange':
    outCol = 'yellow'
else:
    outCol = outCol.lower()

print("\n---- single value output, e.g. formatted in color -----")
print(colored(text=respDict['DrugName'],color=outCol))


---- single value output, e.g. formatted in color -----
phenazone


#### 3.2 substructure searching
Here an example on using substructre searching of a smiles string. This uses a POST call. <br>
(simple print output)

In [30]:
url = "https://www.spotrm.com/api/v1/search/drug/substructure/smiles"
#smiles = "C1CN(CCN1CC2=CC3=C(C=C2)OCO3)C(=O)COC4=CC=C(C=C4)Cl"
smiles = "CC(=O)NC1=CC=C(O)C=C1"
headers = {"content-type": "application/json"}
response = requests.post(url, data=json.dumps(smiles), headers=headers, auth=authData)
respDict = json.loads(response.text)
print(json.dumps(respDict, indent = 2))     

{
  "40": {
    "ChemName": "N-(4-{[2-hydroxy-3-(propan-2-ylamino)propyl]oxy}phenyl)acetamide",
    "ColourCode": "Red",
    "DrugName": "practolol",
    "MarketStatus": "Withdrawn",
    "References": [
      {
        "Ref": "practolol.pdf",
        "RefType": "Mgph"
      }
    ],
    "Synonym": ""
  },
  "76": {
    "ChemName": "N-(4-hydroxyphenyl)acetamide",
    "ColourCode": "Red",
    "DrugName": "paracetamol",
    "MarketStatus": "On the market (2020)",
    "References": [
      {
        "Ref": "paracetamol.pdf",
        "RefType": "Mgph"
      },
      {
        "Ref": "MiC_BQI.pdf",
        "RefType": "MiC"
      }
    ],
    "Synonym": "acetaminophen"
  },
  "96": {
    "ChemName": "4-butyl-1-(4-hydroxyphenyl)-2-phenylpyrazolidine-3,5-dione",
    "ColourCode": "Red",
    "DrugName": "oxyphenbutazone",
    "MarketStatus": "Withdrawn 1985-1986",
    "References": [
      {
        "Ref": "oxyphenbutazone.pdf",
        "RefType": "Mgph"
      }
    ],
    "Synonym": ""
  },
  "

#### 3.3 Obtain only the external reference for above input compound:
The resulting *References* may contain multiple answers, often refering to the PDFs embedded in SPOTRM (currently only viewable only from SPOTRM Web GUI) and are denoted as *Reftype: Mgph* or *MiC*. External linkes are denoted as *Link*. Below example iterates through the resulting JSON and prints only the hyperlink. In Jupyter you can open this linke directly by clicking on it. 

In [98]:
def getRef(refResponse):
    for key, value in refResponse.items():
        for key2, subvalue1 in value.items():
            if (key2 == 'References'):
                for subvalue2 in subvalue1:
                    for refitem, refvalue in subvalue2.items():
                        if(refvalue == 'Link'):
                            print(subvalue1[0]['Ref'])

Since we are using a function, this has to be called first. For more examples on function useage and other API examples, see chapter #4.

In [99]:
getRef(refResponse = respDict)

https://www.ncbi.nlm.nih.gov/books/NBK548853/


### 4. More API examples (using functions)
Below we show some examples of API calls defined via functions; practical for more complex scripts with repeating the same type of calls.<br>
#### All following examples use token based authentication

In [12]:
#
# Retrieve token
#
def getToken():
    url = BASEURL + "/tokens/"
    response = requests.post(url, auth=authData)
    if response.status_code == 200:
        return (json.loads(response.text)["token"], "")
    else:
        print("There was an error: " + json.loads(response.text)["message"])

In [13]:
#
# Query database to find alerts for a smiles string:
# (uses post)
#
def getAlerts(smiles, token):
    url = BASEURL + "/search/smarts/smiles"
    response = requests.post(url, data=json.dumps(smiles), headers=headers, auth=token)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print("There was an error: " + json.loads(response.text)["message"])


#
#  Find drugs associated with an alert ID:
#
def getDrug(alertID, token):
    url = BASEURL + "/get/drug/smarts_id/" + alertID
    response = requests.get(url, auth=token)
    if response.status_code == 200:
        respDict = json.loads(response.text)
        return respDict
    else:
        print("There was an error: " + json.loads(response.text)["message"])

In a Jupyter notebook perhaps somewhat uneccessary would be to defind and use the function `main()`, but we include it here for the sake of typical Python (non-Jupyter) example.<br>
This function specifically finds the SpotRM database ID of a potential alert (here for the smiles **c1ccccc1N(C)C**) then retrieves all drugs associated with that particular alert.

In [14]:
def main():
    token = getToken()
    alerts = getAlerts("c1ccccc1N(C)C", token)
    print("\nThe following alert was found for your structure:")
    for item in alerts:
        print("Alert ID: " + item[0] + ' is "' + item[1] + '"')
    drugs = getDrug(alerts[0][0], token)
    print("\nThe following drugs are associated with this alert:")
    for key in drugs:
        print(drugs[key]["DrugName"])

In [15]:
if __name__ == "__main__":

    main()


The following alert was found for your structure:
Alert ID: 3 is "4-H-Benzene-N-C (not acyl)"

The following drugs are associated with this alert:
nefazodone
trazodone
diclofenac
clozapine
olanzapine
quetiapine
fluperlapine
mianserin
imipramine
erlotinib
phenylbutazone
chlorpromazine
panadiplon
1-Arylpyrazoles
loxapine
mefenamic acid
idelalisib
meclofenamic acid
